In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
# 1. Cargar dataset
data = pd.read_csv("breast_cancer_dataset.csv")
data

In [ ]:
clase = "target" #columna objetivo
X = data.drop(clase, axis=1)
y = data[clase]
# 2. Estandarizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Aplicar PCA con el número máximo de componentes 
pca_full = PCA()
pca_full.fit(X_scaled)
percentage_var = 0.95

# Obtener varianza explicada acumulada

var_acumulada = np.cumsum(pca_full.explained_variance_ratio_)
plt.figure(figsize=(8,5))
plt.plot(range(1, len(var_acumulada)+1), var_acumulada, marker='o')
plt.axhline(y=percentage_var, color='r', linestyle='--', label='95% de varianza')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('Selección del número de componentes en PCA')
plt.xticks(range(1, X.shape[1]+1))
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
n_optimo = np.argmax(var_acumulada >= percentage_var) + 1
print(f"✅ Número mínimo de componentes para explicar el 95% de la varianza: {n_optimo}")

In [ ]:
n_componentes = 2 #n_optimo
pca = PCA(n_components=n_componentes)
X_pca = pca.fit_transform(X_scaled)

# 5. DataFrame transformado
columnas_pca = [f"PC{i+1}" for i in range(n_componentes)]
df_pca = pd.DataFrame(X_pca, columns=columnas_pca)
df_pca["Clase"] = y
print("✅ DataFrame transformado:")
print(df_pca.head())

df_pca.to_csv(f"data_conocidos_pca_{n_componentes}.csv", index=False)
print("✅ DataFrame transformado guardado como data_conocidos_pca.csv")

# 6. División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df_pca.drop("Clase", axis=1), df_pca["Clase"], test_size=0.3, random_state=42, stratify=y
)

# 7. Entrenamiento del clasificador
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

In [ ]:
if n_componentes == 2:
    fig = px.scatter(
        df_pca, x="PC1", y="PC2", color="Clase",
        title="PCA con 2 componentes principales (plotly)",
        labels={"PC1": "Componente Principal 1", "PC2": "Componente Principal 2"},
        width=800, height=600
    )
    fig.update_traces(marker=dict(size=8, opacity=0.7))
    fig.show()

elif n_componentes == 3:
    fig = px.scatter_3d(
        df_pca, x="PC1", y="PC2", z="PC3", color="Clase",
        title="PCA con 3 componentes principales (plotly 3D)",
        labels={"PC1": "Componente Principal 1", "PC2": "Componente Principal 2", "PC3": "Componente Principal 3"},
        width=850, height=700
    )
    fig.update_traces(marker=dict(size=5, opacity=0.7))
    fig.show()

In [ ]:
def clasificar_nuevos_datos(nuevos_datos_df):
    """
    nuevos_datos_df: DataFrame con las 13 columnas originales del dataset de vinos.
    """
    # Verificar columnas
    if list(nuevos_datos_df.columns) != list(X.columns):
        raise ValueError("Las columnas deben coincidir con las originales del dataset.")

    # Estandarizar con el scaler entrenado
    nuevos_escalados = scaler.transform(nuevos_datos_df)

    # Proyectar con el PCA entrenado
    nuevos_pca = pca.transform(nuevos_escalados)

    # Clasificar
    predicciones = clf.predict(nuevos_pca)
    return predicciones

nuevo = pd.read_csv("breast_cancer_dataset_desconocidos.csv")
pred = clasificar_nuevos_datos(nuevo)
print("Predicciones para los nuevos datos:", pred)
nuevo["pred"] = pred
nuevo.to_csv(f"data_desconocidos_con_pred_{n_componentes}.csv", index=False)